**Import Modules**

In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
import csv
import ast

**Load Data**

In [11]:
with open('dataset/recycling_article_embedding_clean.csv', 'r') as csvfile:
  reader = csv.reader(csvfile, delimiter = ',')
    
  next(csvfile)
  judul_artikel = [row[0] for row in reader]
    
  csvfile.seek(0)
  next(csvfile)
  judul_eng = [row[1] for row in reader]
    
  csvfile.seek(0)
  next(csvfile)
  judul_embeddings = [row[2] for row in reader]

judul_embeddings = [np.fromstring(embedding[1:-1], sep=' ') for embedding in judul_embeddings]
judul_embeddings = np.array(judul_embeddings)

print(judul_artikel[0])
print(judul_eng[0])
print(len(judul_embeddings[0]))

Membuat Keset dari Kain Perca dengan Menyulam
Patchwork
512


**Preprocess Title (Remove Stopwords)**

In [12]:
stopword_file_path = "dataset/stopwords-id.txt"

with open(stopword_file_path, 'r') as file:
    stopwords = [line.strip() for line in file]

print(len(stopwords))

757


In [13]:
judul_artikel_filtered = []

for title in judul_artikel:
    words = title.split()
    filtered_words = [word for word in words if word.lower() not in stopwords]
    filtered_title = ' '.join(filtered_words)
    judul_artikel_filtered.append(filtered_title)

print(judul_artikel_filtered)

['Keset Kain Perca Menyulam', 'Tote Bag Kain Perca', 'Sarung Kabel Kain Perca', 'Bandana Kain Perca', 'Keset Kain Perca Kawat Kasa', 'Syal Kaos Bekas', 'Tote Bag Kaos Bekas', 'Gelang Kaos Bekas', 'Kaos Ombre', 'Kantung Koin Kaos Bekas', 'Tas Belanja Kaos Bekas', 'Boneka Kaos Bekas', 'Selimut Kaos Bekas', 'Masker kain perca', 'Tas Jinjing Kain Perca', 'Tas Rias Make Up', 'Selimut kain perca', 'Sarung bantal', 'Sarung kabel', 'Taplak Meja Kain Perca', 'Benang Kaos', 'Bandana kain perca', 'Bros Kain Perca', 'Gantungan Kunci Kain Perca', 'Gorden Kain Perca Sederhana', 'Wadah Unik Kaos Bekas', 'kertas daur ulang', 'Vas Bunga Koran Bekas', 'Jam Dinding Koran', 'Dompet Anyaman Kertas', 'Keranjang Kertas Koran Bekas', 'Mangkok Koran Bekas', 'Kerajinan Koran Kertas Daur Ulang', 'Kerajinan Koran Tempat Pensil', 'Kerajinan Koran Bekas Tong Sampah', 'Jam Dinding Koran Bekas', 'Kerajinan Bunga Kertas Koran', 'Tempat Pensil Koran', 'Tempat Tisu Koran Bekas', 'Tempat Tisu Sederhana Koran Bekas', 'Ala

**Tokenize Title**

In [14]:
vocab_size = 1000
embedding_dim = 16
max_length = 15
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(judul_artikel_filtered)

sequences = tokenizer.texts_to_sequences(judul_artikel_filtered)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type, padding = padding_type)

padded = np.array(padded)

**Create and Train Model**

In [38]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=512, input_length=max_length),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512)
])

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(padded, judul_embeddings, epochs=500)

Epoch 1/500
5/5 [==============================] - 4s 35ms/step - loss: 0.0018
Epoch 2/500
5/5 [==============================] - 0s 33ms/step - loss: 0.0016
Epoch 3/500
5/5 [==============================] - 0s 34ms/step - loss: 0.0015
Epoch 4/500
5/5 [==============================] - 0s 32ms/step - loss: 0.0015
Epoch 5/500
5/5 [==============================] - 0s 33ms/step - loss: 0.0015
Epoch 6/500
5/5 [==============================] - 0s 32ms/step - loss: 0.0015
Epoch 7/500
5/5 [==============================] - 0s 32ms/step - loss: 0.0015
Epoch 8/500
5/5 [==============================] - 0s 32ms/step - loss: 0.0015
Epoch 9/500
5/5 [==============================] - 0s 34ms/step - loss: 0.0015
Epoch 10/500
5/5 [==============================] - 0s 34ms/step - loss: 0.0015
Epoch 11/500
5/5 [==============================] - 0s 33ms/step - loss: 0.0015
Epoch 12/500
5/5 [==============================] - 0s 32ms/step - loss: 0.0015
Epoch 13/500
5/5 [==============================]

Inference Data

In [39]:
judul_embeddings_predict = model.predict(padded)

5/5 [==============================] - 1s 10ms/step


**Compute Similiarity**

In [40]:
from sklearn.metrics.pairwise import cosine_similarity

top_similarities_indices = []

for i, embedding in enumerate(judul_embeddings_predict):
    similarities = cosine_similarity(embedding.reshape(1, -1), judul_embeddings_predict).flatten()

    top_indices = np.argsort(similarities)[-6:-1][::-1]

    top_similarities_indices.append(top_indices)

In [43]:
list_judul_similiar = []

for n, similiar in enumerate(top_similarities_indices):
    judul_similiar = []
    for index in similiar:
        judul_similiar.append(judul_artikel[index])

    list_judul_similiar.append(judul_similiar)
    
    print(f"{judul_artikel[n]}:\n {judul_similiar}")
    print()

Membuat Keset dari Kain Perca dengan Menyulam:
 ['Membuat Keset dari Kain Perca dengan Kawat Kasa', 'Membuat Bros dari Kain Perca', 'Taplak Meja dari Kain Perca', 'Selimut dari kain perca', 'Bandana dari Kain Perca']

Tote Bag dari Kain Perca:
 ['Tas Jinjing dari Kain Perca', 'Selimut dari kain perca', 'Taplak Meja dari Kain Perca', 'Bandana kain perca', 'Bandana dari Kain Perca']

Sarung Kabel dari Kain Perca:
 ['Sarung kabel', 'Taplak Meja dari Kain Perca', 'Membuat Bros dari Kain Perca', 'Selimut dari kain perca', 'Gorden Dari Kain Perca Yang Sederhana']

Bandana dari Kain Perca:
 ['Bandana kain perca', 'Membuat Bros dari Kain Perca', 'Masker dari kain perca', 'Taplak Meja dari Kain Perca', 'Membuat Keset dari Kain Perca dengan Menyulam']

Membuat Keset dari Kain Perca dengan Kawat Kasa:
 ['Membuat Keset dari Kain Perca dengan Menyulam', 'Membuat Bros dari Kain Perca', 'Taplak Meja dari Kain Perca', 'Selimut dari kain perca', 'Bandana dari Kain Perca']

Syal Dari Kaos Bekas:
 ['Seli

In [44]:
print(list_judul_similiar)

[['Membuat Keset dari Kain Perca dengan Kawat Kasa', 'Membuat Bros dari Kain Perca', 'Taplak Meja dari Kain Perca', 'Selimut dari kain perca', 'Bandana dari Kain Perca'], ['Tas Jinjing dari Kain Perca', 'Selimut dari kain perca', 'Taplak Meja dari Kain Perca', 'Bandana kain perca', 'Bandana dari Kain Perca'], ['Sarung kabel', 'Taplak Meja dari Kain Perca', 'Membuat Bros dari Kain Perca', 'Selimut dari kain perca', 'Gorden Dari Kain Perca Yang Sederhana'], ['Bandana kain perca', 'Membuat Bros dari Kain Perca', 'Masker dari kain perca', 'Taplak Meja dari Kain Perca', 'Membuat Keset dari Kain Perca dengan Menyulam'], ['Membuat Keset dari Kain Perca dengan Menyulam', 'Membuat Bros dari Kain Perca', 'Taplak Meja dari Kain Perca', 'Selimut dari kain perca', 'Bandana dari Kain Perca'], ['Selimut Kaos Bekas', 'Tote Bag Kaos Bekas', 'Gelang Kaos Bekas', 'Tas Belanja Kaos Bekas', 'Kantung Koin Dari Kaos Bekas'], ['Tas Belanja Kaos Bekas', 'Kantung Koin Dari Kaos Bekas', 'Gelang Kaos Bekas', 'Tot

**Add Precomputed Top Similiarity Title to Dataset**

In [50]:
input_file_path = 'dataset/recycling_article_dataset.csv'

output_file_path = 'dataset/recycling_article_datasets_computed_relevant.csv'

new_column_data = list_judul_similiar

df = pd.read_csv(input_file_path)

df['Artikel Serupa'] = new_column_data

df.to_csv(output_file_path, index=False, encoding='utf-8-sig')

print(f"Similiar Article Column Written to {output_file_path}")

Similiar Article Column Written to dataset/recycling_article_dataset_precomputed.csv
